<a href="https://colab.research.google.com/github/jj132535/YOLOv8-human-video-detection/blob/main/yolotest_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
video_link = '/content/drive/MyDrive/YOLO video assets/16-9_CHUU_Strawberry.mp4'
# '/content/aespa-Whiplash.mp4'
output_video_path = '/content/output_video_chuu.mp4'

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="BaQK3K31NGd7tpZicwDT")
project = rf.workspace("chris-kydks").project("people-detection-2csbw")
version = project.version(11)
dataset = version.download("yolov8")


In [ ]:
!cat /content/People-Detection-11/data.yaml

In [ ]:
!pip install PyYAML

In [ ]:
import yaml

data = { 'train' : '/content/People-Detection-11/train/images',
         'val' : '/content/People-Detection-11/valid/images',
         'test' : '/content/People-Detection-11/test/images',
          'names' : ['Person'],
          'nc' : 1}

with open('/content/People-Detection-11/test-data.yaml', 'w') as f:
  yaml.dump(data, f)

with open('/content/People-Detection-11/test-data.yaml','r') as f:
  people_data = yaml.safe_load(f)
  display(people_data)

In [ ]:
!pip install ultralytics

In [ ]:
import ultralytics


ultralytics.checks()

In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8n.pt')

In [ ]:
print(type(model.names), len(model.names))

print(model.names)

In [ ]:
model.train(data = '/content/People-Detection-11/test-data.yaml', epochs = 10, batch =32, imgsz = 416)

In [ ]:
print(type(model.names), len(model.names))

print(model.names)

In [ ]:
#results = model.predict(source= video_link, save=True)

In [ ]:
## 영상 리프레이밍

In [ ]:
import cv2
import torch

# 비디오 저장 설정 (세로 비율 9:16)
width, height = 1080, 1920
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, 30, (width, height))

In [ ]:
# YOLO 모델로 비디오 처리 (탐지된 객체 저장 포함)
results = model(source=video_link, stream=True)

In [ ]:
# 초기 설정
frame_count = 0

# 각 프레임에 대해 탐지된 객체를 기준으로 리프레이밍
for r in results:
    frame = r.orig_img  # 원본 프레임 가져오기
    bboxes = r.boxes.xywh if hasattr(r, 'boxes') and r.boxes is not None else None
    frame_count += 1

    if frame_count >= 3:  # 첫 두 프레임은 건너뜀
        if bboxes is not None and bboxes.shape[0] > 0:
            for bbox in bboxes.cpu().numpy():  # numpy 배열로 변환하여 사용
                x_center, y_center, box_width, box_height = bbox

                # 탐지된 객체의 중심을 기준으로 세로로 크롭
                x1 = int(x_center - width / 2)
                x2 = int(x_center + width / 2)
                y1 = 0
                y2 = frame.shape[0]  # 원래 높이를 유지

                # 프레임의 경계를 벗어나는 경우 조정
                x1 = max(0, x1)
                x2 = min(frame.shape[1], x2)

                cropped_frame = frame[y1:y2, x1:x2]  # 세로로 크롭된 프레임
                resized_frame = cv2.resize(cropped_frame, (width, height))
                out.write(resized_frame)  # 비디오에 작성
        else:
            print(f"No detections in frame {frame_count}")


In [ ]:
# VideoWriter 객체 닫기
out.release()

In [ ]:
print("Video reframing complete and saved to:", output_video_path)